In [1]:
from utils.helpers import *

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import warnings
warnings.filterwarnings('ignore')

# Bayesian Modelling of Covid-19

Here we consider two simple models for the growth of the number of cases in various countries: exponential vs sigmoidal. 


The exponential curve continues to increase with no slowdown, while a sigmoidal curve will eventually plateau. By comparing these two simple models we can see if countries are mitigating the effect of Covid-19 within their borders. 


To look at different models, simply:

1. Choose a country 
2. Choose a number of days to forecast (default of 7)
3. Click `Run Interact`

In [2]:
country_list = ['Italy', 'Germany', 'China', 'Spain', 'Canada', 'US', 'Canada British Columbia', 'Canada Alberta', 'Canada Ontario']
country_list.sort()

country_widget= widgets.Dropdown(
    options=country_list,
    value='Italy',
    description='Country:',
    disabled=False,
)

num_days_widget = widgets.IntText(
    value=7,
    step=1,
    description='Number of Future Days:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

ymax_widget = widgets.IntText(
    value=-1,
    description='Y Max:',
    disabled=False
)

out = interact_manual(plot_country, country=country_widget, num_days=num_days_widget, ymax = ymax_widget)
#plot_country('Italy')

interactive(children=(Dropdown(description='Country:', index=6, options=('Canada', 'Canada Alberta', 'Canada B…